# **Thermal Sharpening using a Random Forest Model**

### Thermal Sharpening

* **Thermal sharpening**, also known as **thermal sharpening fusion or pan-sharpening** for
thermal imagery, is a technique used to enhance the spatial resolution of thermal images by integrating them with higher-resolution optical or panchromatic images.



* This process helps in improving the clarity and detail of thermal data, which is often captured at coarser spatial resolutions compared to visible or near-infrared bands.

### Why is Thermal Sharpening Needed?


Thermal sensors on satellites like **Landsat, MODIS, ASTER, and VIIRS** typically have lower spatial resolution compared to optical sensors. For instance:
* **Landsat 8**:

Thermal Infrared Sensor (TIRS) bands have a 100m resolution.
Operational Land Imager (OLI) bands have a 30m resolution.

* **MODIS**:

Thermal bands have a 1000m (1km) resolution, much coarser than optical bands (~250m).

This difference in spatial resolution makes it difficult to analyze thermal variations at finer scales. Thermal sharpening techniques bridge this gap by using high-resolution optical data to enhance thermal imagery.

### Thermal Sharpening Techniques

1. **Regression-Based Methods**

* Uses a statistical relationship between high-resolution optical data (e.g., NDVI, vegetation indices) and thermal data.
* Assumes that surface temperature variations correlate with land cover type.

2. **Wavelet Transform Techniques**

* Applies multi-scale decomposition (wavelet transforms) to blend thermal and optical images while preserving key features.

3. **High-Resolution Visible & Near-Infrared Data (HRV-NIR)**

* Uses high-resolution visible and near-infrared data to guide thermal sharpening.
* Example: Landsat’s 30m optical bands can be used to sharpen its 100m thermal bands.

4. **Hybrid and AI-Based Methods**

* Machine learning (e.g., Random Forest, Neural Networks) and deep learning models improve sharpening accuracy by learning complex relationships between optical and thermal bands.

### Applications of Thermal Sharpening


✅ **Urban Heat Island Studies** – Provides detailed temperature variations in cities.

✅ **Precision Agriculture** – Detects crop stress and irrigation needs at finer scales.

✅ **Wildfire Monitoring** – Enhances fire detection and hotspot mapping.

✅ **Energy Efficiency & Heat Loss Detection** – Assists in identifying poorly insulated buildings.

✅ **Environmental Monitoring** – Tracks thermal pollution in water bodies and land surfaces.

### Random Forest (RF) Model in Remote Sensing

**Random Forest (RF)** is a machine learning algorithm based on ensemble decision trees.

It is widely used in remote sensing applications, including thermal sharpening, due to its ability to handle complex relationships between different spectral bands, vegetation indices, and temperature variations.


**How Random Forest is Applied in Thermal Sharpening?**

Thermal sharpening involves enhancing the spatial resolution of coarse thermal images using higher-resolution optical data (e.g., from visible and near-infrared bands).

Random Forest helps predict high-resolution thermal values by learning patterns between thermal and optical features.

**Key Steps in Using RF for Thermal Sharpening**:
1. **Data Collection and Preprocessing**

* Gather coarse-resolution thermal images (e.g., from Landsat TIRS, MODIS).
* Collect high-resolution optical images (e.g., NDVI, NDBI, SAVI from Landsat OLI or Sentinel-2).
* Resample the optical images to match the target high resolution.

2. **Feature Selection**

* Select key predictors from optical bands and vegetation indices (e.g., NDVI, LST, texture features).
*  The assumption is that land surface temperature correlates with land cover.

3. **Training the Random Forest Model**

* Train the RF model using high-resolution thermal data from another source (e.g., airborne thermal sensors or historical high-resolution datasets) as ground truth.

* Use the selected optical predictors as input features and the coarse-resolution thermal data as the dependent variable.

4. **Prediction & Thermal Sharpening**

* The trained RF model predicts high-resolution thermal values at finer scales.
* The output is an enhanced thermal image that retains realistic temperature variations.

### **Important**

*Apply the logic of 1 Angular Degree = 100Km where applicable. Such a case would be where a Gographic Coordinate System is used for CRS Definition and imposes the use of Lon/Lat Parameters.*

# **T-Sharp Execution**

## Importing Necessary Libraries

In [ ]:
import ee
import geemap
import xarray as xr

In [ ]:
import xee

## Project Initialization

Insert your ProjectID from Earth Engine

In [ ]:
ee.Authenticate()

ee.Initialize(
    project='---------------',
    opt_url='https://earthengine-highvolume.googleapis.com')

## Identification of Area of Interest via Geometry Plotting

In [ ]:
map = geemap.Map(basemap = "SATELLITE")
map

In [ ]:
roi = map.draw_last_feature.geometry()
roi

## Importing MODIS Terra Land Surface Temperature and Emissivity 8-Day Global 1km

The MOD11A2 V6.1 product provides an average 8-day land surface temperature (LST) in a 1200 x 1200 kilometer grid.

Each pixel value in MOD11A2 is a simple average of all the corresponding MOD11A1 LST pixels collected within that 8 day period.

The MOD11A2 does a simple averaging of all daily LST values.

In [ ]:
lst = ee.ImageCollection("MODIS/061/MOD11A2").filterDate('2020', '2021').select('LST_Day_1km').mean().multiply(0.02).subtract(273.15)

## Importing MODIS Terra Vegetation Indices 16-Day Global 250m

The MOD13Q1 V6.1 product provides a Vegetation Index (VI) value at a per pixel basis.

There are two primary vegetation layers. The first is the Normalized Difference Vegetation Index (NDVI) which is referred to as the continuity index to the existing National Oceanic and Atmospheric Administration-Advanced Very High Resolution Radiometer (NOAA-AVHRR) derived NDVI.

The MODIS NDVI product is computed from atmospherically corrected bi-directional surface reflectances that have been masked for water, clouds, heavy aerosols, and cloud shadows.

In [ ]:
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1").filterDate('2020', '2021').select('NDVI').mean().multiply(0.0001)

## Stacking Extracted Land Surface Temperature and NDVI as Bands in a Single Image

In [ ]:
stack = ee.Image.cat([lst, ndvi]).select([0,1], ['lst', 'ndvi'])
stack

## Converting the Newly Stacked Image to an XArray Data Structure at 1 Km Spatial Resolution

In [ ]:
ds1km = xr.open_dataset(stack, engine='ee', crs = 'EPSG:4326', scale = 0.01, geometry = roi )

## Converting the Newly Stacked Image to an XArray Data Structure at 250 m Spatial Resolution

In [ ]:
ds250m = xr.open_dataset(stack, engine='ee', crs = 'EPSG:4326', scale = 0.0025, geometry = roi )

## Converting the 1 Km and 250 m Spatial Resolution Data Structures to Data Frames

In [ ]:
df1km = ds1km.to_dataframe()
df250m = ds250m.to_dataframe()

In [ ]:
df1km

Define ndvi as the predictor variable, x, and lst as the predicted variable, y.

In [ ]:
x = df1km[['ndvi']]
y = df1km['lst']

In [ ]:
from sklearn.model_selection import train_test_split

## Split the Training and Testing Data in a 95:5 Percent Ratio

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size = 0.05,
                                                    random_state = 42)

## Import the Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(x_train, y_train)

## Predicting Land Surface Temperature at 1 Km Spatial Resolution using the Testing Predictor Variable i.e., NDVI

In [ ]:
y_pred = model.predict(x_test)

## Compute the Mean Square Error, Root Mean Square Error and R Squared for the RF Model Above

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import numpy as np

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f'RMSE:{rmse}')
print(f'R2:{r2}')

## Add a new variable to the 250 m Spatial Resolution Data Frame and assign the NDVI at 250 m resolution as the Predictor Variable to the Model created above whose metrics we just computed

In [ ]:
df250m['lst_sharp'] = model.predict(df250m[['ndvi']])

In [ ]:
df250m

## Convert the Data Frame at 250 m Spatial Resolution Back to an XArray Data Structure at 250 m Spatial Resolution to for visualization

In [ ]:
ds250 = df250m.to_xarray()

In [ ]:
ds250

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (8,3))
plt.tight_layout(w_pad=4)

ds250.lst.plot(x = 'lon', y = 'lat', robust = True, cmap = 'hot', ax = ax[0])
ds250.lst_sharp.plot(x = 'lon', y = 'lat', robust = True, cmap = 'hot', ax = ax[1])

ax[0].set_title('1000 meters')
ax[1].set_title('250 meters')

#plt.savefig('thermalsharpening.png', dpi = 360, bbox_inches = 'tight')